# Building a simulation with Orca from the ground up

Sam Maurer, UrbanSim User Meeting, Nov. 4, 2016

### Import Orca and its new testing framework

In [1]:
import orca
import orca_test as ot
from orca_test import OrcaSpec, TableSpec, ColumnSpec

### Import NumPy and Pandas libraries, for data work

In [2]:
import numpy as np
import pandas as pd

### Register some tables with our baseline data

In [3]:
@orca.table(cache=True)
def parks():
    return pd.read_csv('data/parks.csv')

@orca.table(cache=True)
def zones():
    return pd.read_csv('data/zones.csv')

### Look at the data

In [4]:
print orca.get_table('parks').to_frame()

         name  size  trees  acc_cbd  acc_res  budget
0    Lakeside    50   1050     0.45     0.55    6000
1       Plaza     2     50     0.20     0.80    3000
2  Greenfield   200   2800     0.90     0.15   10000


In [5]:
print orca.get_table('zones').to_frame()

          name   pop  empl
0     Downtown   900  2000
1  Residential  6000   150


### Register model steps for population, employment, budget growth

In [6]:
@orca.step()
def zone_transition(zones):
    zones['pop'] *= [1.05, 1]
    zones['empl'] *= [1.05, 1]
    return
    
@orca.step()
def parks_transition(parks, year):
    if year < 2018:
        parks['budget'] -= 1000
    return

### Register a model step to predict visitors to parks

In [7]:
@orca.step()
def predict_visitors(parks, zones, year):
    
    # Calculate weighted accessibilities
    parks['pop_access'] = parks.acc_cbd * zones.pop[0] + parks.acc_res * zones.pop[1]
    parks['empl_access'] = parks.acc_cbd * zones.empl[0] + parks.acc_res * zones.empl[1]

    # Predict visitors
    vis = 0.15 * parks.pop_access + 1.5 * parks.empl_access + 0.5 * np.log(parks.budget)
    
    # Save the output
    parks['visitors_' + str(year)] = np.round(vis)
    return

### Loose ends: Define the "year" variable

In [8]:
@orca.injectable()
def year():
    return orca.get_injectable('iter_var')

### Test the baseline data (new Orca functionality!)

In [9]:
def test_data():
    ot.assert_orca_spec(OrcaSpec('',
        TableSpec('zones',
            ColumnSpec('pop', min=0, missing=False),
            ColumnSpec('empl', min=0, missing=False)),
        TableSpec('parks',
            ColumnSpec('acc_cbd', min=0, max=1, missing=False),
            ColumnSpec('acc_res', min=0, max=1, missing=False),
            ColumnSpec('budget', min=0.01, missing=False))))

try:
    test_data()
    print "All clear!"

except Exception, e:
    print e

All clear!


### Run the model

In [10]:
%%capture
orca.clear_cache()
years = range(2016, 2021)

orca.run(['zone_transition', 'parks_transition', 'predict_visitors'], iter_vars=years)

### Look at the output

In [11]:
orca.get_table('parks').to_frame()

,name,size,trees,acc_cbd,acc_res,budget,pop_access,empl_access,visitors_2016,visitors_2017,visitors_2018,visitors_2019,visitors_2020
0,Lakeside,50,1050,0.45,0.55,4000,3816.894033,1231.153406,2104.0,2178.0,2256.0,2338.0,2423.0
1,Plaza,2,50,0.20,0.80,1000,5029.730681,630.512625,1562.0,1595.0,1629.0,1666.0,1704.0
2,Greenfield,200,2800,0.90,0.15,8000,1933.788066,2319.806813,3136.0,3284.0,3439.0,3603.0,3774.0
